In [ ]:
# BOtCH: улучшенный чат-бот с генеративной моделью DialoGPT
В этом ноутбуке реализован бот с DialoGPT, базовая оценка качества и демонстрация.


In [ ]:
!pip install transformers torch nltk matplotlib --quiet


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import nltk
import matplotlib.pyplot as plt
import random

nltk.download('punkt')


In [ ]:
model_name = 'microsoft/DialoGPT-small'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)


In [ ]:
def generate_response(history, user_input, max_length=1000):
    new_user_input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors='pt')
    bot_input_ids = torch.cat([history, new_user_input_ids], dim=-1) if history is not None else new_user_input_ids
    output_ids = model.generate(bot_input_ids, max_length=max_length, pad_token_id=tokenizer.eos_token_id)
    response_ids = output_ids[:, bot_input_ids.shape[-1]:]
    response_text = tokenizer.decode(response_ids[0], skip_special_tokens=True)
    return output_ids, response_text


In [ ]:
history = None
user_inputs = [
    "Привет!",
    "Как тебя зовут?",
    "Расскажи шутку",
    "Спасибо, пока!"
]

for user_input in user_inputs:
    history, bot_response = generate_response(history, user_input)
    print(f"Пользователь: {user_input}")
    print(f"БОТ: {bot_response}")
    print('---')


In [ ]:
import nltk
nltk.download('punkt')  # если не скачано
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import time

# Загружаем модель и токенизатор (пример с DialoGPT small)
model_name = "microsoft/DialoGPT-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

def generate_response_with_metrics(tokenizer, model, chat_history_ids, user_input, reference=None):
    new_input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors='pt')
    if chat_history_ids is not None:
        bot_input_ids = torch.cat([chat_history_ids, new_input_ids], dim=-1)
    else:
        bot_input_ids = new_input_ids

    start = time.time()
    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)
    end = time.time()
    gen_time = end - start

    response = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
    response_length = len(response)

    bleu_score = None
    if reference is not None:
        smoothie = SmoothingFunction().method4
        bleu_score = sentence_bleu([nltk.word_tokenize(reference.lower())], nltk.word_tokenize(response.lower()), smoothing_function=smoothie)

    return chat_history_ids, response, gen_time, response_length, bleu_score

test_data = [
    ("Привет!", "Привет! Как я могу помочь?"),
    ("Как тебя зовут?", "Я бот, меня зовут BOtCH."),
    ("Расскажи шутку", "Почему программисты любят природу? Потому что там много багов!"),
    ("Пока", "До свидания!")
]

bleu_scores = []
chat_history_ids = None  # для хранения контекста, если нужно

for question, reference in test_data:
    chat_history_ids, response, gen_time, resp_len, bleu = generate_response_with_metrics(tokenizer, model, chat_history_ids, question, reference)
    bleu_scores.append(bleu)
    print(f"Вопрос: {question}")
    print(f"Ответ бота: {response}")
    print(f"Эталон: {reference}")
    print(f"BLEU: {bleu:.3f}")
    print(f"Время генерации: {gen_time:.3f} сек")
    print(f"Длина ответа: {resp_len} символов")
    print('---')


In [ ]:
import time
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

# Функция генерации с метриками
def generate_response_with_metrics(tokenizer, model, chat_history_ids, user_input, reference=None):
    new_input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors='pt')
    if chat_history_ids is not None:
        bot_input_ids = torch.cat([chat_history_ids, new_input_ids], dim=-1)
    else:
        bot_input_ids = new_input_ids

    start = time.time()
    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)
    end = time.time()
    gen_time = end - start

    response = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
    response_length = len(response)

    bleu_score = None
    if reference is not None:
        # Разбиваем на слова, считаем BLEU с сглаживанием
        smoothie = SmoothingFunction().method4
        bleu_score = sentence_bleu([reference.split()], response.split(), smoothing_function=smoothie)

    return chat_history_ids, response, gen_time, response_length, bleu_score

# Пример вызова (reference — эталонный ответ, если есть)
chat_history_ids, bot_response, gen_time, resp_len, bleu = generate_response_with_metrics(tokenizer, model, chat_history_ids, user_input, reference="эталонный ответ")

print(f"Ответ: {bot_response}")
print(f"Время генерации: {gen_time:.2f} сек")
print(f"Длина ответа: {resp_len} символов")
if bleu is not None:
    print(f"BLEU: {bleu:.4f}")


In [ ]:
plt.figure(figsize=(8,4))
plt.bar(range(len(bleu_scores)), bleu_scores, tick_label=[q for q, _ in test_data])
plt.title('BLEU scores for test questions')
plt.ylabel('BLEU score')
plt.xticks(rotation=45)
plt.ylim(0, 1)
plt.show()
